In [46]:
import nbformat
import os
from nbconvert.preprocessors import ExecutePreprocessor

In [47]:
# Paths to look for .ipynb
# DIRS = ['../../pandas-basic/teilnehmer/loesung/',]
DIRS = ['../tests']

In [48]:
# Options
LAST_ROW_STANDING = True # delete all successive code cell but leave one for starting
SPECIFIC_VERSION = 1.1
REMOVE_FIRST = True # no 

In [49]:
# Functions
def open_nb(filepath):
    with open(filepath, encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
        
    return nb

def save_nb(filepath, nb):
    # check if dir exist
    if not os.path.isdir(os.path.dirname(filepath) + '/clean'):
        os.mkdir(os.path.dirname(filepath) + '/clean')
    
    # my files have a ' - Loesung' at the end, which is to remove
    with open(os.path.dirname(filepath) + '/clean/' + os.path.basename(filepath).replace(' - Loesung', ''), 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
        
def process_cells(nb):
    cells = []
    last_cell_cleared = REMOVE_FIRST
    
    # ToDo: Add Version
    if SPECIFIC_VERSION > 0:
        nb.metadata['practise_cell_export_version'] = SPECIFIC_VERSION
    else:
        nb.metadata['practise_cell_export_version'] = \
            nb.metadata.get('practise_cell_export_version', 0) + 1
    
    for c in nb.get('cells'):
        if c.get('cell_type') == 'code':
            # check metadata if cell is protected
            if c.get('metadata', False) and \
            c.get('metadata').get('ht_protected', False):

                cells.append(c)
                last_cell_cleared = False

            else:
                # decide if cell will be removed
                if not last_cell_cleared:
                    cells.append(nbformat.v4.new_code_cell())
                    last_cell_cleared = True

        else:
            if c.get('metadata', False) and \
            c.get('metadata').get('ht_enforce_remove', False):
                # remove add cell if enforced
                if not last_cell_cleared:
                    cells.append(nbformat.v4.new_markdown_cell())
                    last_cell_cleared = True
            else:
                cells.append(c)
                last_cell_cleared = False
      
    nb['cells'] = cells
    return nb

In [51]:
for d in DIRS:
    for entry in os.scandir(d):
        if entry.is_file() and entry.name.endswith(".ipynb"):
            nb = open_nb(entry.path)
            nb = process_cells(nb)
            save_nb(entry.path, nb)
            nb = None
    
    print('Done Converting Files: {}'.format(entry.path))
            


Done Converting Files: ../tests\test_file_1.ipynb
